# COMP 3610 Assignment 1: NYC Yellow Taxi Data Pipeline & Visualization Dashboard

---

## Student ID- 816039310


## Imports

In [84]:
import os
import sys
import requests
from pathlib import Path

import polars as pl
import pandas as pd
import duckdb
import plotly.express as px
import plotly.graph_objects as go

---
# Part 1: Data Ingestion



### 1.1: Programmatic Download


In [85]:

RAW_DIR = Path('data/raw')
RAW_DIR.mkdir(parents=True, exist_ok=True)

URLS = {
    'yellow_tripdata_2024-01.parquet': (
        'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet'
    ),
    'taxi_zone_lookup.csv': (
        'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
    ),
}

def download_file(url: str, dest: Path, chunk_size: int = 8192) -> None:
    if dest.exists():
        print(f'  [SKIP] {dest.name} already exists - skipping download.')
        return

    print(f'  [DOWNLOAD] {dest.name} …', end=' ', flush=True)
    response = requests.get(url, stream=True, timeout=120)

    if response.status_code != 200:
        raise RuntimeError(
            f'Failed to download {url}. HTTP status: {response.status_code}'
        )

    with open(dest, 'wb') as fh:
        for chunk in response.iter_content(chunk_size=chunk_size):
            fh.write(chunk)

    size_mb = dest.stat().st_size / (1024 ** 2)
    print(f'done ({size_mb:.1f} MB)')


print('Downloading required data files …')
for filename, url in URLS.items():
    download_file(url, RAW_DIR / filename)

print('\nAll files are present in data/raw/')

  [SKIP] yellow_tripdata_2024-01.parquet already exists - skipping download.
  [SKIP] taxi_zone_lookup.csv already exists - skipping download.

All files are present in data/raw/


### 1.2: Data Validation



In [86]:
TRIP_FILE  = RAW_DIR / 'yellow_tripdata_2024-01.parquet'
ZONE_FILE  = RAW_DIR / 'taxi_zone_lookup.csv'

raw_df = pl.read_parquet(TRIP_FILE)

REQUIRED_COLUMNS = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'total_amount',
    'payment_type',
]
#Verify all expected columns exist in the dataset
missing_cols = [c for c in REQUIRED_COLUMNS if c not in raw_df.columns]
if missing_cols:
    raise ValueError(f'Validation FAILED - missing columns: {missing_cols}')
print('Column check passed - all required columns present.')

#Check that date columns are valid datetime types
date_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
for col in date_cols:
    dtype = raw_df[col].dtype
    if not isinstance(dtype, (pl.Datetime)):
        raise ValueError(
            f'Validation FAILED - column "{col}" has unexpected dtype {dtype}. '
            'Expected a Datetime type.'
        )
print('DateTime type check passed - pickup/dropoff columns are proper Datetime.')

#Report total row count and print a summary to the console
total_rows = raw_df.shape[0]
MIN_EXPECTED_ROWS = 100000
if total_rows < MIN_EXPECTED_ROWS:
    raise ValueError(
        f'Validation FAILED: only {total_rows:,} rows found. '
        f'Expected at least {MIN_EXPECTED_ROWS:,}.'
    )
print(f'Row count check passed - {total_rows:,} rows loaded.')

print('\nRaw Dataset Summary')
print(f'Rows    : {total_rows:>10,}')
print(f'Columns : {raw_df.shape[1]:>10}')
print('\nSchema:')
for name, dtype in zip(raw_df.columns, raw_df.dtypes):
    print(f'{name:<35} {dtype}')

Column check passed - all required columns present.
DateTime type check passed - pickup/dropoff columns are proper Datetime.
Row count check passed - 2,964,624 rows loaded.

Raw Dataset Summary
Rows    :  2,964,624
Columns :         19

Schema:
VendorID                            Int32
tpep_pickup_datetime                Datetime(time_unit='ns', time_zone=None)
tpep_dropoff_datetime               Datetime(time_unit='ns', time_zone=None)
passenger_count                     Int64
trip_distance                       Float64
RatecodeID                          Int64
store_and_fwd_flag                  String
PULocationID                        Int32
DOLocationID                        Int32
payment_type                        Int64
fare_amount                         Float64
extra                               Float64
mta_tax                             Float64
tip_amount                          Float64
tolls_amount                        Float64
improvement_surcharge               Float6

---
# Part 2: Data Transformation & Analysis


### 2.1: Data Cleaning


In [87]:
CRITICAL_COLS = [
    'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'PULocationID', 'DOLocationID', 'fare_amount',
]

rows_Before = raw_df.shape[0]

#Removing rows with null values in critical columns (pickup/dropoff
#times, locations, fare)
df = raw_df.drop_nulls(subset=CRITICAL_COLS)
removed_nulls = rows_Before - df.shape[0]

#Filtering out invalid trips: trips with zero or negative distance,
#negative fares, or fares exceeding $500
rows_before = df.shape[0]
df = df.filter(pl.col('trip_distance') > 0)
removed_dist = rows_before - df.shape[0]

#Filtering out invalid trips: trips withnegative fares, or fares exceeding $500
rows_before = df.shape[0]
df = df.filter((pl.col('fare_amount') > 0) & (pl.col('fare_amount') <= 500))
removed_fare = rows_before - df.shape[0]

#Removing trips where dropoff time is before pickup time
rows_before = df.shape[0]
df = df.filter(pl.col('tpep_dropoff_datetime') > pl.col('tpep_pickup_datetime'))
removed_time = rows_before - df.shape[0]

rows_after = df.shape[0]
total_removed = rows_Before - rows_after
#Documenting how many rows were removed and why
print('Data Cleaning Summary')
print(f'Rows before cleaning                        : {rows_Before:>10,}')
print(f'Removed - null critical columns             : {removed_nulls:>10,}')
print(f'Removed - zero or negative distance         : {removed_dist:>10,}')
print(f'Removed - invalid fare                      : {removed_fare:>10,}')
print(f'Removed - dropoff time before pickup time   : {removed_time:>10,}')
print(f'\nTotal removed                               : {total_removed:>10,}  ({100*total_removed/rows_Before:.2f}%)')
print(f'Rows after cleaning                         : {rows_after:>10,}')

Data Cleaning Summary
Rows before cleaning                        :  2,964,624
Removed - null critical columns             :          0
Removed - zero or negative distance         :     60,371
Removed - invalid fare                      :     34,569
Removed - dropoff time before pickup time   :        112

Total removed                               :     95,052  (3.21%)
Rows after cleaning                         :  2,869,572


In [88]:
#Sample 100,000 rows for manageable analysis
df = df.sample(n=100000, seed=42)

#a sample was taken as 3 million rows is a lot to visualize (rightfully said by sir in class)


### 2.2: Feature Engineering


In [89]:
df = df.with_columns([
    #Hour of day (0-23)
    pl.col('tpep_pickup_datetime').dt.hour().alias('pickup_hour'),

    #Day of week (1=Monday, 7=Sunday)
    pl.col('tpep_pickup_datetime').dt.to_string('%A').alias('pickup_day_of_week'),

    #Trip duration in minutes
    ((pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime'))
        .dt.total_seconds() / 60
    ).alias('trip_duration_minutes'),
])

#cannot put speed in above, as duration needs to exist first
df = df.with_columns([
    pl.when(pl.col('trip_duration_minutes') > 0)
      .then(pl.col('trip_distance') / (pl.col('trip_duration_minutes') / 60))
      .otherwise(0.0)
      .alias('trip_speed_mph'),
])

print('Feature engineering complete. New columns added:',
      ['pickup_hour', 'pickup_day_of_week', 'trip_duration_minutes', 'trip_speed_mph'])


Feature engineering complete. New columns added: ['pickup_hour', 'pickup_day_of_week', 'trip_duration_minutes', 'trip_speed_mph']


### 2.3: SQL Analysis with DuckDB


In [90]:
con = duckdb.connect()

con.register('trips', df.to_arrow())

zones_df = pd.read_csv(ZONE_FILE)
con.register('zones', zones_df)

print('Tables registered:')
print(con.execute("SHOW TABLES").df())

Tables registered:
    name
0  trips
1  zones


#### Query 1: Top 10 Busiest Pickup Zones

**Question:** What are the top 10 busiest pickup zones by total number of trips?
This tells us the most demand-dense areas of the city, so that TLC and fleet operators know which areas need the most taxis available at any given time.

In [91]:
q1 = """
SELECT
    z.Zone            AS pickup_zone,
    z.Borough         AS borough,
    COUNT(*)          AS total_trips
FROM trips t
JOIN zones z ON t.PULocationID = z.LocationID
GROUP BY z.Zone, z.Borough
ORDER BY total_trips DESC
LIMIT 10
"""

top_pickup_zones = con.execute(q1).df()
print('Q1: Top 10 Busiest Pickup Zones')
print(top_pickup_zones.to_string(index=False))

Q1: Top 10 Busiest Pickup Zones
                 pickup_zone   borough  total_trips
                 JFK Airport    Queens         4913
       Upper East Side South Manhattan         4842
              Midtown Center Manhattan         4835
       Upper East Side North Manhattan         4711
                Midtown East Manhattan         3568
Penn Station/Madison Sq West Manhattan         3560
   Times Sq/Theatre District Manhattan         3551
         Lincoln Square East Manhattan         3523
           LaGuardia Airport    Queens         3097
       Upper West Side South Manhattan         3007


#### Query 2: Average Fare by Hour of Day

**Question:** What is the average fare amount for each hour of the day?
This reveals the relationship between time, traffic, and trip length and allows us to see that fares are expected to be higher during peak hours (4am to 6am) possibly due to the morning rush to get to work. This is useful for drivers deciding when to work and for riders budgeting their trips.

In [102]:
q2 = """
SELECT
    pickup_hour,
    ROUND(AVG(fare_amount), 2)     AS avg_fare,
FROM trips
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

fare_by_hour = con.execute(q2).df()
print('Q2: Average Fare by Hour of Day')
print(fare_by_hour.to_string(index=False))

Q2: Average Fare by Hour of Day
 pickup_hour  avg_fare
           0     19.52
           1     17.83
           2     16.85
           3     18.16
           4     23.09
           5     26.85
           6     22.33
           7     19.21
           8     18.12
           9     17.89
          10     18.17
          11     17.61
          12     17.64
          13     18.71
          14     19.60
          15     19.37
          16     19.46
          17     18.00
          18     16.97
          19     17.51
          20     18.41
          21     18.02
          22     19.14
          23     20.75


#### Query 3: Payment Type Distribution

**Question:** What percentage of trips use each payment type?
Understanding payment preferences helps TLC and fleet operators plan infrastructure (e.g., card terminal maintenance),  what proportion of revenue is trackable electronically vs cash.

In [93]:
q3 = """
SELECT
    CASE payment_type
        WHEN 1 THEN 'Credit Card'
        WHEN 2 THEN 'Cash'
        WHEN 3 THEN 'No Charge'
        WHEN 4 THEN 'Dispute'
        WHEN 5 THEN 'Unknown'
        ELSE 'Other'
    END                                                         AS payment_label,
    COUNT(*)                                                    AS trips,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2)         AS pct
FROM trips
GROUP BY payment_type
ORDER BY trips DESC
"""

payment_dist = con.execute(q3).df()
print('Q3: Payment Type Distribution')
print(payment_dist.to_string(index=False))

Q3: Payment Type Distribution
payment_label  trips   pct
  Credit Card  79901 79.90
         Cash  14908 14.91
        Other   4074  4.07
      Dispute    763  0.76
    No Charge    354  0.35


#### Query 4: Average Tip Percentage by Day of Week (Credit Card Only)

**Question:** What is the average tip percentage (tip_amount/fare_amount) by day of week, for
credit card payments only?
Do riders tip more generously on certain days? Weekend nights might see higher tips from nightlife passengers. This is relevant to driver earnings and scheduling decisions.

In [94]:
q4 = """
SELECT
    pickup_day_of_week,
    ROUND(AVG(tip_amount / NULLIF(fare_amount, 0)) * 100, 2)  AS avg_tip_pct,
    COUNT(*)                                                    AS trips
FROM trips
WHERE payment_type = 1          -- credit card only
  AND fare_amount  > 0
GROUP BY pickup_day_of_week
ORDER BY
    -- Order Monday→Sunday using a CASE for correct weekday ordering
    CASE pickup_day_of_week
        WHEN 'Monday'    THEN 0
        WHEN 'Tuesday'   THEN 1
        WHEN 'Wednesday' THEN 2
        WHEN 'Thursday'  THEN 3
        WHEN 'Friday'    THEN 4
        WHEN 'Saturday'  THEN 5
        WHEN 'Sunday'    THEN 6
    END
"""

tip_by_dow = con.execute(q4).df()
print('Q4: Average Tip % by Day of Week (Credit Card Only)')
print(tip_by_dow.to_string(index=False))

Q4: Average Tip % by Day of Week (Credit Card Only)
pickup_day_of_week  avg_tip_pct  trips
            Monday        25.66  10777
           Tuesday        25.66  12284
         Wednesday        25.72  13717
          Thursday        25.60  11624
            Friday        25.55  11033
          Saturday        25.02  11342
            Sunday        25.08   9124


#### Query 5: Top 5 Most Common Pickup-Dropoff Zone Pairs

**Question:** What are the top 5 most common pickup-dropoff zone pairs?
This reveals the most heavily used routes and can guide decisions on taxi dispatching, surge pricing, and understanding commuter vs tourist travel patterns.

In [95]:
q5 = """
SELECT
    pu.Zone  AS pickup_zone,
    dropoff.Zone  AS dropoff_zone,
    COUNT(*) AS trips
FROM trips t
JOIN zones pu ON t.PULocationID = pu.LocationID
JOIN zones dropoff ON t.DOLocationID = dropoff.LocationID
GROUP BY pu.Zone, dropoff.Zone
ORDER BY trips DESC
LIMIT 5
"""

top_od_pairs = con.execute(q5).df()
print('Q5: Top 5 Pickup-Dropoff Zone Pairs')
print(top_od_pairs.to_string(index=False))

Q5: Top 5 Pickup-Dropoff Zone Pairs
          pickup_zone          dropoff_zone  trips
Upper East Side South Upper East Side North    793
Upper East Side North Upper East Side South    681
Upper East Side North Upper East Side North    517
Upper East Side South Upper East Side South    502
       Midtown Center Upper East Side South    328


---
# Part 3: Dashboard Development



### 3.1: Visualization 1: Top 10 Pickup Zones (Bar Chart)

In [96]:
fig1 = px.bar(
    top_pickup_zones,
    x='total_trips',
    y='pickup_zone',
    orientation='h',
    color='borough',
    labels={'total_trips': 'Number of Trips', 'pickup_zone': 'Taxi Zone'},
    title='Top 10 NYC Taxi Pickup Zones: January 2024',
    text='total_trips',
)
fig1.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig1.update_layout(
    yaxis=dict(autorange='reversed'),
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    legend_title='Borough',
    height=420,
)
fig1.show()

**Interpretation:** Midtown Manhattan zones (eg: Upper East Side and Midtown Center) dominate pickup activity, whereas zones like Upper West Side South and Lincoln Square East have the least pickups. This confirms that business districts and major transit hubs are the primary demand generators for yellow taxis.

### 3.2: Visualization 2: Average Fare by Hour of Day (Line Chart)

In [97]:
fig2 = px.line(
    fare_by_hour,
    x='pickup_hour',
    y='avg_fare',
    markers=True,
    labels={'pickup_hour': 'Hour of Day (0 = midnight)', 'avg_fare': 'Average Fare ($)'},
    title='Average Taxi Fare by Hour of Day: January 2024',
)
fig2.update_traces(line_color='steelblue', marker_size=7)
fig2.update_layout(
    xaxis=dict(
        tickmode='linear',
        dtick=1,
        range=[0, 23],
        fixedrange=True,
    ),
    height=400,
)
fig2.show()

**Interpretation:** Fares are highest in the early morning hours (4–6 AM), likely because of the early morning rush to get to work. The midday dip (11 AM–2 PM) corresponds to shorter cross-town trips during the lunch window. Evening fares rise again as commuters head home.

### 3.3: Visualization 3: Trip Distance Distribution (Histogram)

In [101]:
fig3 = px.histogram(
        df, x="trip_distance",
        range_x=[0, 20],
        labels={"trip_distance": "Trip Distance (miles)"},
        title="Trip Distance Distribution: January 2024",
        color_discrete_sequence=["steelblue"],
    )
fig3.update_traces(xbins=dict(start=0, end=20, size=0.5))
fig3.update_layout(
    bargap=0.05,
    height=400,
    yaxis_title="Number of Trips",
    xaxis=dict(tickmode="linear", dtick=2),
)
fig3.update_layout(bargap=0.05, height=400)
fig3.show()

**Interpretation:** The distribution is strongly right-skewed; the vast majority of NYC yellow cab trips are under 5 miles, reflecting short intra-Manhattan travel. There is a long tail of longer trips ranging from 6 miles to 20 miles (likely JFK/LaGuardia airport runs) which pull the mean well above the median. This means the mean fare overstates the usual trip cost experienced by most riders. This skew is typical of urban taxi datasets worldwide.

### 3.4: Visualization 4: Payment Type Breakdown (Pie Chart)

In [99]:
fig4 = px.bar(
    payment_dist,
    x='payment_label',
    y='trips',
    title='Payment Type Breakdown - January 2024',
    labels={'payment_label': 'Payment Type', 'trips': 'Number of Trips'},
    text='trips',
    color='payment_label',
    color_discrete_sequence=px.colors.qualitative.Set2,
)
fig4.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig4.update_layout(
    height=420,
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        range=[0, payment_dist['trips'].max() * 1.15],
    ),
    xaxis=dict(showgrid=False),
)
fig4.show()

**Interpretation:** Credit card is by far the dominant payment method, reflecting the near-universal adoption of card terminals in NYC taxis. Cash still accounts for a notable minority, suggesting that a segment of riders, potentially tourists or older passengers prefer or require cash payment.

### 3.5: Visualization 5: Trips by Day of Week and Hour (Heatmap)

In [100]:
dow_hour = con.execute("""
    SELECT
        pickup_day_of_week,
        pickup_hour,
        COUNT(*) AS trips
    FROM trips
    GROUP BY pickup_day_of_week, pickup_hour
""").df()

DOW_ORDER = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
pivot = (
    dow_hour
    .pivot(index='pickup_day_of_week', columns='pickup_hour', values='trips')
    .reindex(DOW_ORDER)
)

fig5 = px.imshow(
    pivot,
    labels=dict(x='Hour of Day', y='Day of Week', color='Trip Count'),
    title='NYC Taxi Trips by Day of Week and Hour - January 2024',
    color_continuous_scale='YlOrRd',
    aspect='auto',
)
fig5.update_layout(height=380)
fig5.show()

**Interpretation:** The heatmap reveals two distinct demand patterns. Weekdays show a classic bimodal commuter pattern with peaks during morning (7–9 AM) and evening (5–8 PM) rush hours. Weekends (particularly Friday and Saturday nights) show a very different pattern: demand is concentrated in late evening (10 PM–2 AM), corresponding to entertainment and nightlife travel. The early morning hours (3–5 AM) are consistently the quietest period across all days.

# AI Tools Used
##Claude AI
Claude was mainly used to fix errors in my Plotly code cells and to format certain pandas used expressions to polars (Lab 2, which I based some of this Assignment on, used pandas, so I used Claude to 'translate' it to polars)

All submitted work reflects my own understanding of the material. I am prepared to explain any line of code or SQL query during office hours if asked.